In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow
from tensorflow import keras
import scipy as sc
from scipy import signal
import cv2 
import PIL

In [2]:
df = pd.read_csv(r"E:\Academic\Project\User Identity from Time-Frequency Analysis\50_Users_Acceleromter_Data.csv")
df

,X,Y,Z,Label
0,0.340509,8.308413,4.140585,0
1,0.381370,8.390134,4.249548,0
2,0.272407,8.471856,4.018002,0
3,0.149824,8.430995,4.290409,0
4,0.272407,8.430995,4.481094,0
...,...,...,...,...
749995,-7.201759,-4.750096,-1.532289,49
749996,-7.354988,-5.209783,-1.991976,49
749997,-7.201759,-5.056554,-2.145205,49
749998,-7.354988,-5.363012,-2.298433,49


In [3]:
length = 15000

In [4]:
import time
start_time = time.time()
N_Users=50
k=0
X=[0 for p in range(N_Users)] # Final Array
for i in range(0,length*N_Users,length):
    temp=[]
    for j in range(length):
        x_y_z = [df.iloc[i+j,0],df.iloc[i+j,1],df.iloc[i+j,2]]
        temp.append(x_y_z)
    X[k]=temp
    k+=1 #incrementing [Final Array] indices
print("--- %s seconds ---" % (time.time() - start_time))

--- 14.76772689819336 seconds ---


In [5]:
t=np.array(X)
t.shape

(50, 15000, 3)

In [6]:
# Main Algo for Creating Windows .....#1

def create(df,wind_size,hop_size,m_index):
  #l=((len(df.x)-wind_size)/hop_size)+1
  rows, cols = (wind_size, 3) 
#  a = [[0 for i in range(cols)] for j in range(rows)] 
  array= [0 for i in range(int((length-wind_size)/hop_size)+1)]
  k = 0
  for i in range(0,(length-wind_size+1),hop_size):
      a=create1(df,i,wind_size,m_index)
       #array.append(a)
      array[k] = a
      k+=1
  #return pd.DataFrame(array)
  return array


In [15]:
# Main Algo for Creating Windows .....#2

def create1(df,x,w,m_index):
     rows = w 
     cols = 3
     arr = [[0 for i in range(cols)] for j in range(rows)] 
     for i in range(0,w,1):
        arr[i][0]=df[x+i][0]
        arr[i][1]=df[x+i][1]
        arr[i][2]=df[x+i][2]
     arr = np.array(arr)
     f1,t1,x = signal.stft(arr[:,0],5,nperseg=10)
     f2,t2,y = signal.stft(arr[:,1],5,nperseg=10)
     f3,t3,z = signal.stft(arr[:,2],5,nperseg=10)
     temp_a = np.concatenate((abs(np.array(x)),abs(np.array(y)),abs(np.array(z))),axis=0)
     return temp_a.flatten()

In [16]:
import time
start_time = time.time()
X_final = [0 for i in range( N_Users)]
for m_index in range(N_Users):
  X_final[m_index]=create(X[m_index],225,10,m_index)
print("--- %s seconds ---" % (time.time() - start_time))

--- 50.904356956481934 seconds ---


In [17]:
t=np.array(X_final[0])
t.shape             # The shape gives(number of windows,shape of TF plot image)

(1478, 828)

In [18]:
X_input=[]
for i in range(N_Users):
  temp = X_final[i]
  X_input += temp
X_input = np.array(X_input)

In [19]:
y = [0 for i in range(X_input.shape[0])]
for i in range(X_input.shape[0]):
  y[i]=int(i*N_Users/X_input.shape[0])
y = np.array(y)

In [20]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import tree
from sklearn import ensemble
from sklearn import svm
from sklearn import metrics
X_train , X_test , y_train , y_test = train_test_split(X_input,y,test_size = 0.2,shuffle=True)

In [21]:
clf_Logistic_Reg = sklearn.linear_model.LogisticRegression()
clf_SVM = sklearn.svm.SVC()
clf_dt = sklearn.tree.DecisionTreeClassifier()
clf_rf = sklearn.ensemble.RandomForestClassifier()
clf_knn = sklearn.neighbors.KNeighborsClassifier()

In [22]:
import time
start_time = time.time()
clf_Logistic_Reg.fit(X_train,y_train)
print("--- %s seconds ---" % (time.time() - start_time))

C:\Users\Saptarshi\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Saptarshi\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


--- 5817.729847431183 seconds ---


C:\Users\Saptarshi\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [23]:
y_pred_train_Log = clf_Logistic_Reg.predict(X_train)
y_pred_test_Log = clf_Logistic_Reg.predict(X_test)

In [24]:
acc_train = metrics.accuracy_score(y_train,y_pred_train_Log)
acc_test = metrics.accuracy_score(y_test,y_pred_test_Log)
print('Logistic Regression:')
print('Accuracy: %.4f' % acc_train,'val_acc: %.4f' %acc_test)

Logistic Regression:
Accuracy: 0.4029 val_acc: 0.2668


In [25]:
recall = metrics.recall_score(y_test, y_pred_test_Log, labels=[1,2], average='micro')
precision = metrics.precision_score(y_test, y_pred_test_Log, labels=[1,2], average='micro')
f1 = metrics.f1_score(y_test, y_pred_test_Log, labels=[1,2], average='micro')
print('Recall: %.4f' % recall,'Precision: %.4f' % precision,'F-Measure: %.4f' % f1)

Recall: 0.1228 Precision: 0.1166 F-Measure: 0.1196


In [26]:
start_time = time.time()
clf_rf.fit(X_train,y_train)
print("--- %s seconds ---" % (time.time() - start_time))

C:\Users\Saptarshi\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


--- 30.34790325164795 seconds ---


In [27]:
y_pred_train_rf = clf_rf.predict(X_train)
y_pred_test_rf = clf_rf.predict(X_test)

In [28]:
acc_train = metrics.accuracy_score(y_train,y_pred_train_rf)
acc_test = metrics.accuracy_score(y_test,y_pred_test_rf)
print('Random Forest:')
print('Accuracy: %.4f' % acc_train,'val_acc: %.4f' %acc_test)

Random Forest:
Accuracy: 0.9979 val_acc: 0.4635


In [29]:
recall = metrics.recall_score(y_test, y_pred_test_rf, labels=[1,2], average='micro')
precision = metrics.precision_score(y_test,y_pred_test_rf, labels=[1,2], average='micro')
f1 = metrics.f1_score(y_test, y_pred_test_rf, labels=[1,2], average='micro')
print('Recall: %.4f' % recall,'Precision: %.4f' % precision,'F-Measure: %.4f' % f1)

Recall: 0.4291 Precision: 0.2125 F-Measure: 0.2842
